In [1]:
import pathlib
import tempfile

import geopandas
import pystac
import rioxarray
import rioxarray.merge
import stac2dcache

from shapely.geometry import Polygon
from stac2dcache.utils import get_asset
from eratosthenes.generic.handler_www import get_tar_file

# Retile DEM according to Sentinel-2 scheme

In [21]:
catalog_url = ("https://webdav.grid.surfsara.nl:2880"
               "/pnfs/grid.sara.nl/data/eratosthenes/"
               "disk/red-glacier_sentinel-2")
collection_id = "sentinel-s2-l1c"
dem_index_url = (
    "https://webdav.grid.surfsara.nl:2880"
    "/pnfs/grid.sara.nl/data/eratosthenes/"
    "disk/GIS/Elevation/ArcticDEM_Tile_Index_Rel7_10m.geojson"
)
tmp_path = "./"
dem_tiles_url = (
    "https://webdav.grid.surfsara.nl:2880"
    "/pnfs/grid.sara.nl/data/eratosthenes/"
    "disk/ArcticDEM_tiles_sentinel-2"
)

In [3]:
# configure connection to dCache
dcache = stac2dcache.configure(
    filesystem="dcache", 
    token_filename="macaroon.dat"
)

In [4]:
# read DEM index
with dcache.open(dem_index_url) as f:
    dem_index = geopandas.read_file(f)

In [5]:
def read_catalog(url):
    """
    Read STAC catalog from URL
    
    :param url: urlpath to the catalog root
    :return: PySTAC Catalog object
    """
    url = url if url.endswith("catalog.json") else f"{url}/catalog.json"
    catalog = pystac.Catalog.from_file(url)
    return catalog

In [6]:
# read image catalog
catalog = read_catalog(catalog_url)
subcatalog = catalog.get_child(collection_id)

In [7]:
TILE_ID_KEYS = [
    "sentinel:utm_zone", 
    "sentinel:latitude_band", 
    "sentinel:grid_square"
] 
def get_sentinel2_tile_id(item):
    """
    Construct the tile ID for a Sentinel-2 STAC item
    
    :param item: PySTAC Item object
    :return: tile ID
    """
    return "".join([
        str(item.properties[k]) for k in TILE_ID_KEYS
    ])

In [10]:
# loop over catalog, look for all the tiles presents
tiles = {}
for item in subcatalog.get_all_items():
    tile_id = get_sentinel2_tile_id(item)
    if tile_id not in tiles:
        tiles[tile_id] = item
tiles

{'5VMG': <Item id=S2B_5VMG_20210329_0_L1C>}

In [18]:
def get_itersecting_DEM_tile_URLs(index, geometry):
    """ 
    Find the DEM tiles that intersect the geometry and extract the
    URLs from the index. NOTE: the geometries need to be in the 
    same CRS!
    
    :param index: DEM tile index (GeoDataFrame)
    :param geometry: shapely geometry object
    :returl: URL array
    """
    mask = index.intersects(tile_geometry)
    index = index[mask]
    return index.fileurl

In [19]:
def download_and_mosaic(url_list, tmp_path, bbox, crs, transform):
    """
    Download DEM tiles and create mosaic according to satellite imagery tiling
    scheme
    
    :param url_list: list of DEM tile URLs
    :param tmp_path: work path where to download and untar DEM tiles
    :param bbox: bound box (xmin, ymin, xmax, ymax)
    :param crs: coordinate reference system of the DEM tile
    :param transform: Affine transform of the DEM tile
    :return: retiled DEM (DataArray object) 
    """
    with tempfile.TemporaryDirectory(dir=tmp_path) as tmpdir:
    
        for url in url_list:
            get_tar_file(url, tmpdir)

        dem_tiles_filename = pathlib.Path(tmpdir).glob("*_dem.tif")
        dem_tiles = [rioxarray.open_rasterio(f) for f in dem_tiles_filename]

        # merge DEM tiles
        dem = rioxarray.merge.merge_arrays(dem_tiles)

        # reproject to image CRS
        dem_reproj = dem.rio.reproject(crs, transform=transform)

        # crop area within tile
        dem_clip = dem_reproj.rio.clip_box(*bbox)
    
    return dem_clip

In [20]:
# loop over identified tiles
for tile_id, item in tiles.items():

    da = get_asset(
        catalog,
        asset_key="B02",
        item_id=item.id,
        filesystem=dcache,
        load=False
    )
    bbox = da.rio.bounds()
    crs = da.spatial_ref.crs_wkt
    transform = da.rio.transform()
    
    tile_geometry = Polygon.from_bounds(*bbox)
    urls = get_itersecting_DEM_tile_URLs(
        dem_index.to_crs(crs),
        tile_geometry
    )
    
    dem = download_and_mosaic(urls, tmp_path, bbox, crs, transform)
    
    # save raster file and upload it
    output_file = f"{tile_id}.tif"
    dem.rio.to_raster(output_file)
    dcache.upload(output_file, f"{dem_tiles_url}/{output_file}")

GDAL headers saved to: /var/folders/t6/r2gjczqj7bb8798wr4g1p87m0000gn/T/tmpdhfo07i6
